In [1]:
import numpy as np 
import torch
torch.manual_seed(16)
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms

In [2]:
tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

In [3]:
training_set = datasets.MNIST('/root', download=True, train=True, transform = tf)
test_set = datasets.MNIST('/root', download=True, train=False, transform=tf)

train_load = torch.utils.data.DataLoader(training_set,batch_size = 64, shuffle = True)
test_load = torch.utils.data.DataLoader(test_set,batch_size = 64, shuffle = True)

In [4]:
#validate image size 28x28
image,label = iter(train_load).next()

In [5]:
#without transform of mean and variance
temp_data = datasets.MNIST('/root',download = True, train=True,transform = transforms.Compose
                           ([transforms.ToTensor()]))
temp_load = torch.utils.data.DataLoader(temp_data,batch_size = 64)
ti, tl = iter(temp_load).next()

**Neural Network Architechture:**

Input: 28x28=784 \\
hidden layer 1: 128 (ReLu) \\
hidden layer 2: 64 (ReLu) \\
output layer: 10 (softmax)



In [6]:
model = nn.Sequential(nn.Linear(784, 128), nn.ReLU(),
                      nn.Linear(128,64),nn.ReLU(),
                      nn.Linear(64, 10), nn.LogSoftmax(dim=1))
loss_type = nn.NLLLoss()

In [7]:
opt = optim.SGD(model.parameters(),lr = 0.004, momentum = 0.96)
n_epochs = 16
for i in range(n_epochs):
    temp_loss = 0
    for images, labels in train_load:
        fl = nn.Flatten()
        images = fl(images)
        #images = images.view(images.shape[0],-1)
        opt.zero_grad()
        output = model(images)
        loss = loss_type(output,labels)
        loss.backward()
        opt.step()
        temp_loss += loss.item()
    else:
        print("epoch {} - training loss {}".format(i,temp_loss/len(train_load)))


epoch 0 - training loss 0.469388761889261
epoch 1 - training loss 0.18148292344746622
epoch 2 - training loss 0.12898942023547474
epoch 3 - training loss 0.10596180251185923
epoch 4 - training loss 0.08647015409321705
epoch 5 - training loss 0.0744619833869931
epoch 6 - training loss 0.06504754512912102
epoch 7 - training loss 0.05885382726200735
epoch 8 - training loss 0.05447160910024667
epoch 9 - training loss 0.04682027330055638
epoch 10 - training loss 0.03993788021841277
epoch 11 - training loss 0.0394517939724177
epoch 12 - training loss 0.03920398659882263
epoch 13 - training loss 0.031417427019114104
epoch 14 - training loss 0.028743071698095663
epoch 15 - training loss 0.027050572875503654


In [8]:
correct, all = 0,0
for image, label in test_load:
    for i in range(len(label)):
        temp_image = image[i].view(1,784) 
        assert list(temp_image.shape) == [1,784] #using Flatten() gives Tensor.Size([16,784])
        with torch.no_grad():
            log_prob = model(temp_image)
        prob = torch.exp(log_prob)
        class_out = list(prob.numpy()[0])
        pred = class_out.index(max(class_out))
        true = label.numpy()[i]
        if true == pred: correct += 1
        all += 1

print('model accuracy = ', (correct/all))


model accuracy =  0.9786
